# Machine Learning Titanic

## Carregando Bibliotecas & os Dados

In [1]:
import pandas as pd
import os
import numpy as np
import sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


In [2]:


# Define o caminho relativo
file_path = os.path.join('..', 'Data', 'processed_data.csv')  # '..' sobe um nível de diretório

# Carrega os dados
df = pd.read_csv(file_path)
df

,passengerid,survived,pclass,name,age,sibsp,parch,ticket,fare,cabin,...,is_male,cabin_group,title,last_name,family_size,fare_group,is_alone,ticket_length,ticket_qtd,sex_pclass_interaction
0,1,0,3,Owen Harris,22.0,1,0,A/5 21171,7.2500,Unk,...,1,Unk,Mr,Braund,2,Low Fare,0,9,1,male_3
1,2,1,1,John Bradley (Florence Briggs Thayer),38.0,1,0,PC 17599,71.2833,C85,...,0,C,Mrs,Cumings,2,High Fare,0,8,1,female_1
2,3,1,3,Laina,26.0,0,0,STON/O2. 3101282,7.9250,Unk,...,0,Unk,Miss,Heikkinen,1,Medium-Low Fare,1,16,1,female_3
3,4,1,1,Jacques Heath (Lily May Peel),35.0,1,0,113803,53.1000,C123,...,0,C,Mrs,Futrelle,2,High Fare,0,6,2,female_1
4,5,0,3,William Henry,35.0,0,0,373450,8.0500,Unk,...,1,Unk,Mr,Allen,1,Medium-Low Fare,1,6,1,male_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Juozas,27.0,0,0,211536,13.0000,Unk,...,1,Unk,Rev,Montvila,1,Medium-Low Fare,1,6,1,male_2
887,888,1,1,Margaret Edith,19.0,0,0,112053,30.0000,B42,...,0,B,Miss,Graham,1,Medium-High Fare,1,6,1,female_1
888,889,0,3,"Catherine Helen ""Carrie""",13.5,1,2,W./C. 6607,23.4500,Unk,...,0,Unk,Miss,Johnston,4,Medium-High Fare,0,10,2,female_3
889,890,1,1,Karl Howell,26.0,0,0,111369,30.0000,C148,...,1,C,Mr,Behr,1,Medium-High Fare,1,6,1,male_1


In [3]:
df.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked_c', 'embarked_q', 'embarked_s',
       'age_group', 'is_male', 'cabin_group', 'title', 'last_name',
       'family_size', 'fare_group', 'is_alone', 'ticket_length', 'ticket_qtd',
       'sex_pclass_interaction'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   passengerid             891 non-null    int64  
 1   survived                891 non-null    int64  
 2   pclass                  891 non-null    int64  
 3   name                    891 non-null    object 
 4   age                     891 non-null    float64
 5   sibsp                   891 non-null    int64  
 6   parch                   891 non-null    int64  
 7   ticket                  891 non-null    object 
 8   fare                    891 non-null    float64
 9   cabin                   891 non-null    object 
 10  embarked_c              891 non-null    int64  
 11  embarked_q              891 non-null    int64  
 12  embarked_s              891 non-null    int64  
 13  age_group               891 non-null    object 
 14  is_male                 891 non-null    in

In [5]:
df.isna().sum()

passengerid                0
survived                   0
pclass                     0
name                       0
age                        0
sibsp                      0
parch                      0
ticket                     0
fare                       0
cabin                      0
embarked_c                 0
embarked_q                 0
embarked_s                 0
age_group                  0
is_male                    0
cabin_group                0
title                      0
last_name                  0
family_size                0
fare_group                15
is_alone                   0
ticket_length              0
ticket_qtd                 0
sex_pclass_interaction     0
dtype: int64

In [6]:
df[df['fare_group'].isna()]

,passengerid,survived,pclass,name,age,sibsp,parch,ticket,fare,cabin,...,is_male,cabin_group,title,last_name,family_size,fare_group,is_alone,ticket_length,ticket_qtd,sex_pclass_interaction
179,180,0,3,Lionel,36.000000,0,0,LINE,0.0,Unk,...,1,Unk,Mr,Leonard,1,NaN,1,4,4,male_3
263,264,0,1,William,40.000000,0,0,112059,0.0,B94,...,1,B,Mr,Harrison,1,NaN,1,6,1,male_1
271,272,1,3,William Henry,25.000000,0,0,LINE,0.0,Unk,...,1,Unk,Mr,Tornquist,1,NaN,1,4,4,male_3
277,278,0,2,"Francis ""Frank""",33.904119,0,0,239853,0.0,Unk,...,1,Unk,Mr,Parkes,1,NaN,1,6,3,male_2
302,303,0,3,William Cahoone Jr,19.000000,0,0,LINE,0.0,Unk,...,1,Unk,Mr,Johnson,1,NaN,1,4,4,male_3
413,414,0,2,Alfred Fleming,33.904119,0,0,239853,0.0,Unk,...,1,Unk,Mr,Cunningham,1,NaN,1,6,3,male_2
466,467,0,2,William,33.904119,0,0,239853,0.0,Unk,...,1,Unk,Mr,Campbell,1,NaN,1,6,3,male_2
481,482,0,2,"Anthony Wood ""Archie""",33.904119,0,0,239854,0.0,Unk,...,1,Unk,Mr,Frost,1,NaN,1,6,1,male_2
597,598,0,3,Alfred,49.000000,0,0,LINE,0.0,Unk,...,1,Unk,Mr,Johnson,1,NaN,1,4,4,male_3
633,634,0,1,William Henry Marsh,39.000000,0,0,112052,0.0,Unk,...,1,Unk,Mr,Parr,1,NaN,1,6,1,male_1


In [7]:

# Mapeamento para a coluna 'age_group'
age_group_mapping = {
    'Baby': 1, 'Child': 2, 'Pre-Teen': 3, 'Teenager': 4,
    'Young Adult': 5, 'Adult': 6, 'Senior': 7, 'Elderly': 8
}
df['age_group'] = df['age_group'].map(age_group_mapping)
df['age_group'] = df['age_group'].fillna(0)

# Mapeamento para a coluna 'cabin_group'
cabin_group_mapping = {
    'Unk': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4,
    'E': 5, 'F': 6, 'G': 7, 'T': 8
}
df['cabin_group'] = df['cabin_group'].map(cabin_group_mapping)
df['cabin_group'] = df['cabin_group'].fillna(0)

# Mapeamento para a coluna 'title'
title_mapping = {
    'Mr': 1, 'Mrs': 2, 'Miss': 3, 'Master': 4, 'Don': 5,
    'Rev': 6, 'Dr': 7, 'Mme': 8, 'Ms': 9, 'Major': 10,
    'Lady': 11, 'Sir': 12, 'Mlle': 13, 'Col': 14, 'Capt': 15,
    'the Countess': 16, 'Jonkheer': 17
}
df['title'] = df['title'].map(title_mapping)
df['title'] = df['title'].fillna(0)

# Mapeamento para a coluna 'fare_group'
fare_group_mapping = {
    'Low Fare': 1, 'Medium-Low Fare': 2, 'Medium-High Fare': 3, 'High Fare': 4
}
df['fare_group'] = df['fare_group'].map(fare_group_mapping)
df['fare_group'] = df['fare_group'].fillna(0)

# Mapeamento para a coluna 'sex_pclass_interaction'
sex_pclass_mapping = {
    'male_3': 1, 'female_1': 2, 'female_3': 3, 'male_1': 4,
    'female_2': 5, 'male_2': 6
}
df['sex_pclass_interaction'] = df['sex_pclass_interaction'].map(sex_pclass_mapping)
df['sex_pclass_interaction'] = df['sex_pclass_interaction'].fillna(0)

In [8]:
df.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked_c', 'embarked_q', 'embarked_s',
       'age_group', 'is_male', 'cabin_group', 'title', 'last_name',
       'family_size', 'fare_group', 'is_alone', 'ticket_length', 'ticket_qtd',
       'sex_pclass_interaction'],
      dtype='object')

In [9]:
#usar so colunas de grupo e nao a original caso tenha grupo
features = [ 'pclass', 'age', 'sibsp', 'parch',
        'embarked_c', 'embarked_q', 'embarked_s',
       'age_group', 'is_male', 'cabin_group', 'title',
       'family_size', 'fare_group', 'is_alone', 'ticket_length', 'ticket_qtd',
       'sex_pclass_interaction']
target = 'survived'

# Codificar variáveis categóricas usando One-Hot Encoding
X = df[features]
#X = pd.get_dummies(df[features], drop_first=True)
y = df[target]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# CRIAR ESSEMBLE DE QUALIDade
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# Definindo os modelos base
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svm', SVC(probability=True))
]

# Definindo o modelo meta (usado para combinar as previsões)
stacking_model = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

# Treinando e avaliando o modelo Stacking
stacking_model.fit(X_train, y_train)
y_pred_stacking = stacking_model.predict(X_test)

print("Acurácia Stacking:", accuracy_score(y_test, y_pred_stacking))


Acurácia Stacking: 0.8268156424581006


# Criando o arquivo de submissao

In [11]:
# Adiciona o diretório 'Scripts' ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','Scripts')))
from preprocessing import preprocess_data
from test_feature import engineer_features

# Define o caminho relativo
file_path = os.path.join('..', 'Data', 'test.csv')  # '..' sobe um nível de diretório

# Carrega os dados
df_test = pd.read_csv(file_path)

df_test = preprocess_data(df_test)
df_test = engineer_features(df_test)

df_test

,passengerid,pclass,name,age,sibsp,parch,ticket,fare,cabin,embarked_c,...,is_male,cabin_group,title,last_name,family_size,fare_group,is_alone,ticket_length,ticket_qtd,sex_pclass_interaction
0,892,3,James,34.500,0,0,330911,7.8292,Unk,0,...,1,Unk,Mr,Kelly,1,Low Fare,1,6,1,male_3
1,893,3,James (Ellen Needs),47.000,1,0,363272,7.0000,Unk,0,...,0,Unk,Mrs,Wilkes,2,Low Fare,0,6,1,female_3
2,894,2,Thomas Francis,62.000,0,0,240276,9.6875,Unk,0,...,1,Unk,Mr,Myles,1,Medium-Low Fare,1,6,1,male_2
3,895,3,Albert,27.000,0,0,315154,8.6625,Unk,0,...,1,Unk,Mr,Wirz,1,Medium-Low Fare,1,6,1,male_3
4,896,3,Alexander (Helga E Lindqvist),22.000,1,1,3101298,12.2875,Unk,0,...,0,Unk,Mrs,Hirvonen,3,Medium-Low Fare,0,7,1,female_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,Woolf,25.125,0,0,A.5. 3236,8.0500,Unk,0,...,1,Unk,Mr,Spector,1,Medium-Low Fare,1,9,1,male_3
414,1306,1,Fermina,39.000,0,0,PC 17758,108.9000,C105,1,...,0,C,Dona,Oliva y Ocana,1,High Fare,1,8,1,female_1
415,1307,3,Simon Sivertsen,38.500,0,0,SOTON/O.Q. 3101262,7.2500,Unk,0,...,1,Unk,Mr,Saether,1,Low Fare,1,18,1,male_3
416,1308,3,Frederick,25.125,0,0,359309,8.0500,Unk,0,...,1,Unk,Mr,Ware,1,Medium-Low Fare,1,6,1,male_3


In [16]:


# Mapeamento para a coluna 'age_group'
age_group_mapping = {
    'Baby': 1, 'Child': 2, 'Pre-Teen': 3, 'Teenager': 4,
    'Young Adult': 5, 'Adult': 6, 'Senior': 7, 'Elderly': 8
}
df_test['age_group'] = df_test['age_group'].map(age_group_mapping)
df_test['age_group'] = df_test['age_group'].fillna(1)

# Mapeamento para a coluna 'cabin_group'
cabin_group_mapping = {
    'Unk': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4,
    'E': 5, 'F': 6, 'G': 7, 'T': 8
}
df_test['cabin_group'] = df_test['cabin_group'].map(cabin_group_mapping)
df_test['cabin_group'] = df_test['cabin_group'].fillna(0)

# Mapeamento para a coluna 'title'
title_mapping = {
    'Mr': 1, 'Mrs': 2, 'Miss': 3, 'Master': 4, 'Don': 5,
    'Rev': 6, 'Dr': 7, 'Mme': 8, 'Ms': 9, 'Major': 10,
    'Lady': 11, 'Sir': 12, 'Mlle': 13, 'Col': 14, 'Capt': 15,
    'the Countess': 16, 'Jonkheer': 17
}
df_test['title'] = df_test['title'].map(title_mapping)
df_test['title'] = df_test['title'].fillna(0)

# Mapeamento para a coluna 'fare_group'
fare_group_mapping = {
    'Low Fare': 1, 'Medium-Low Fare': 2, 'Medium-High Fare': 3, 'High Fare': 4
}
df_test['fare_group'] = df_test['fare_group'].map(fare_group_mapping)
df_test['fare_group'] = df_test['fare_group'].astype(float)  # Convert to float
df_test['fare_group'] = df_test['fare_group'].fillna(0)
# Mapeamento para a coluna 'sex_pclass_interaction'
sex_pclass_mapping = {
    'male_3': 1, 'female_1': 2, 'female_3': 3, 'male_1': 4,
    'female_2': 5, 'male_2': 6
}
df_test['sex_pclass_interaction'] = df_test['sex_pclass_interaction'].map(sex_pclass_mapping)
df_test['sex_pclass_interaction'] = df_test['sex_pclass_interaction'].fillna(0)

In [17]:
df_test.isna().sum()

passengerid               0
pclass                    0
name                      0
age                       3
sibsp                     0
parch                     0
ticket                    0
fare                      1
cabin                     0
embarked_c                0
embarked_q                0
embarked_s                0
age_group                 0
is_male                   0
cabin_group               0
title                     0
last_name                 0
family_size               0
fare_group                0
is_alone                  0
ticket_length             0
ticket_qtd                0
sex_pclass_interaction    0
dtype: int64

In [18]:
df_test[df_test['fare'].isna()]

,passengerid,pclass,name,age,sibsp,parch,ticket,fare,cabin,embarked_c,...,is_male,cabin_group,title,last_name,family_size,fare_group,is_alone,ticket_length,ticket_qtd,sex_pclass_interaction
152,1044,3,Thomas,25.728127,0,0,3701,NaN,Unk,0,...,1,0,1.0,Storey,1,0.0,1,4,1,1


In [19]:
df_test['fare'] = df_test['fare'].fillna(0)

In [20]:

df_test['age'] = df_test['age'].fillna(df_test['age'].mean())

In [21]:
# Adiciona o diretório 'Scripts' ao sys.path

#X_test_final = pd.get_dummies(df_test[features], drop_first=True)

# Alinhar as colunas do conjunto de teste com as do conjunto de treino
X_test_final = df_test.reindex(columns=X.columns, fill_value=0)

# Fazer previsões com o modelo treinado
predictions = stacking_model.predict(X_test_final)

# Criar o DataFrame de submissão
submission_df = pd.DataFrame({
    'PassengerId': df_test['passengerid'],  # Certifique-se de que `PassengerId` está no df_test
    'Survived': predictions
})

# Salvar o arquivo de submissão como CSV
submission_df.to_csv('C:\\Users\\Guilherme\\OneDrive\\Área de Trabalho\\submission4.csv', index=False)

print("Arquivo de submissão criado com sucesso!")

Arquivo de submissão criado com sucesso!
